In [ ]:
%%loadFromPOM
<dependency>
    <groupId>com.google.guava</groupId>
    <artifactId>guava</artifactId>
    <version>28.2-jre</version>
</dependency>

## 1



In [ ]:
import com.google.common.base.CharMatcher;
import com.google.common.base.Joiner;
import com.google.common.base.Splitter;
import com.google.common.collect.Lists;
import com.google.common.primitives.Ints;

int[] concat = Ints.concat(new int[]{1, 2}, new int[]{3, 4, 5});
List<Integer> list = Ints.asList(concat);
System.out.println(Ints.max(concat));

final Joiner joiner = Joiner.on(",").skipNulls();
final Splitter splitter = Splitter.on(",").trimResults().omitEmptyStrings();
final CharMatcher digitMatcher = CharMatcher.anyOf("1234567890");

String join = joiner.join(Lists.newArrayList("a", null, "b"));
System.out.println("join1:" + join);


In [ ]:
import com.google.common.base.Optional;
import com.google.common.cache.Cache;
import com.google.common.cache.CacheBuilder;
import com.google.common.cache.CacheLoader;
import com.google.common.cache.LoadingCache;

import java.util.concurrent.ExecutionException;

/**
 * 模拟耗时操作
 *
 * @param userId
 * @return username
 */
private String getUserName(String userId) {
    System.out.println("执行耗时操作......");
    String username = null;

    switch (userId) {
        case "0101":
            username = "foo";
            break;
        case "0102":
            username = "bar";
            break;
    }

    try {
        Thread.sleep(1000);
    } catch (Exception e) {
        e.printStackTrace();
    }
    return Optional.fromNullable(username).or("");
}

// private Cache<String, String> poiCache = CacheBuilder.newBuilder()
//         .maximumSize(1024 * 1024 * 1024)
//         .build();

private LoadingCache<String, String> poiCache1 = CacheBuilder.newBuilder()
        .maximumSize(1000)
        .build(
                new CacheLoader<String, String>() {
                    @Override
                    public String load(String userId) {
                        return getUserName(userId);
                    }
                });

public String getUserNameById(final String userId) {
    String username = null;
    /// 缓存定义形式1
//     try {
//         username = poiCache.get(userId, () -> getUserName(userId));
//     } catch (ExecutionException e) {
//         e.printStackTrace();
//     }

    /// 缓存定义形式2
    try {
        username = poiCache1.get(userId);
    } catch (ExecutionException e) {
        e.printStackTrace();
    }
    return username;
}

// 后两次循环没有执行方法, 直接取缓存
for (int i = 0; i < 3; i++) {
    System.out.println("--- " + i + " ---");
    System.out.println(getUserNameById("0101"));
    System.out.println(getUserNameById("0102"));
    System.out.println(getUserNameById("0103"));
    System.out.println();
}